In [5]:
from wordcloud import WordCloud, get_single_color_func, STOPWORDS
import plotly.express as px
from PIL import Image
import numpy as np
import os

class SimpleGroupedColorFunc(object):
    """Create a color function object which assigns EXACT colors
       to certain words based on the color to words mapping

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.word_to_color = {word: color
                              for (color, words) in color_to_words.items()
                              for word in words}

        self.default_color = default_color

    def __call__(self, word, **kwargs):
        return self.word_to_color.get(word, self.default_color)


class GroupedColorFunc(object):
    """Create a color function object which assigns DIFFERENT SHADES of
       specified colors to certain words based on the color to words mapping.

       Uses wordcloud.get_single_color_func

       Parameters
       ----------
       color_to_words : dict(str -> list(str))
         A dictionary that maps a color to the list of words.

       default_color : str
         Color that will be assigned to a word that's not a member
         of any value from color_to_words.
    """

    def __init__(self, color_to_words, default_color):
        self.color_func_to_words = [
            (get_single_color_func(color), set(words))
            for (color, words) in color_to_words.items()]

        self.default_color_func = get_single_color_func(default_color)

    def get_color_func(self, word):
        """Returns a single_color_func associated with the word"""
        try:
            color_func = next(
                color_func for (color_func, words) in self.color_func_to_words
                if word in words)
        except StopIteration:
            color_func = self.default_color_func

        return color_func

    def __call__(self, word, **kwargs):
        return self.get_color_func(word)(word, **kwargs)


In [6]:
# Read the whole text.
text = open('D:/LO_resources/bible_quotes_only.csv', encoding='utf-8').read()
# read the mask image
mask = np.array(Image.open("D:/LO_resources/speech_bubble_mask_white.png"))

In [7]:
stopwords = set(STOPWORDS)
stopwords.add("will")

wc = WordCloud(width=2000,height=1000, scale=4,mask=mask,background_color="#000000" )
    #background_color="white", max_words=2000, mask=mask, stopwords=stopwords, contour_width=3, contour_color='white')

# generate word cloud
wc.generate(text)


#f8f1e5 white
#2f3131 black
#f0810f orange
#426e86 blue
#f9ba32 yellow
color_to_words = {
    # words below will be colored
    '#f0810f': ['Moses','David','Abraham','Israelites'],
    '#426e86': ['Israel','Jerusalem','Egypt'],
    '#f9ba32': ['O God','LORD','Christ', 'Lord', 'God', 'GOD','Jesus Christ', 'Lord Jesus', 'Jesus','O LORD', 'Lord GOD']
}

# Words that are not in any of the color_to_words values
# will be colored with a single color function
default_color = '#ffffff' #'#f8f1e5'

# Create a color function with multiple tones
grouped_color_func = GroupedColorFunc(color_to_words, default_color)

# Apply our color function
wc.recolor(color_func=grouped_color_func)

CPU times: total: 3min 5s
Wall time: 3min 9s


In [ ]:
# show
import plotly.express as px
fig = px.imshow(wc)
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

In [8]:
# Export as PNG
image = wc.to_image()
image.save("prayer_speech_bubble_custom_colours.png","PNG")

In [ ]:
# Export as svg (changes font)
#with open("Output.svg", "w") as text_file:
#    text_file.write(wc.to_svg())